In [1]:
import pandas as pd
import numpy as np
import six
import time

from google.cloud import translate_v2 as translate
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/home/erik_hou/translate-api.json')

In [2]:
translate_client = translate.Client(credentials=credentials)

In [3]:
results = translate_client.get_languages()

for language in results:
    print(u'{name} ({language})'.format(**language))

Afrikaans (af)
Albanian (sq)
Amharic (am)
Arabic (ar)
Armenian (hy)
Azerbaijani (az)
Basque (eu)
Belarusian (be)
Bengali (bn)
Bosnian (bs)
Bulgarian (bg)
Catalan (ca)
Cebuano (ceb)
Chichewa (ny)
Chinese (Simplified) (zh)
Chinese (Traditional) (zh-TW)
Corsican (co)
Croatian (hr)
Czech (cs)
Danish (da)
Dutch (nl)
English (en)
Esperanto (eo)
Estonian (et)
Filipino (tl)
Finnish (fi)
French (fr)
Frisian (fy)
Galician (gl)
Georgian (ka)
German (de)
Greek (el)
Gujarati (gu)
Haitian Creole (ht)
Hausa (ha)
Hawaiian (haw)
Hebrew (iw)
Hindi (hi)
Hmong (hmn)
Hungarian (hu)
Icelandic (is)
Igbo (ig)
Indonesian (id)
Irish (ga)
Italian (it)
Japanese (ja)
Javanese (jw)
Kannada (kn)
Kazakh (kk)
Khmer (km)
Kinyarwanda (rw)
Korean (ko)
Kurdish (Kurmanji) (ku)
Kyrgyz (ky)
Lao (lo)
Latin (la)
Latvian (lv)
Lithuanian (lt)
Luxembourgish (lb)
Macedonian (mk)
Malagasy (mg)
Malay (ms)
Malayalam (ml)
Maltese (mt)
Maori (mi)
Marathi (mr)
Mongolian (mn)
Myanmar (Burmese) (my)
Nepali (ne)
Norwegian (no)
Odia (Oriya)

In [6]:
file_to_tran = "text-level/EN_Laptop_Text_Train_Complete_NLI_M.csv"

In [7]:
df = pd.read_csv(file_to_tran, names = ["sent_id", "polarity", "entity-aspect", "text"], delimiter = "\t")

In [8]:
df_unduped = df[["sent_id", "text"]].drop_duplicates(keep = "first")

In [9]:
test_df = df_unduped.iloc[0:8,:]

In [10]:
test_df

,sent_id,text
0,348,Most everything is fine with this machine: spe...
198,285,"I love the size, keyboard, the functions. I do..."
396,337,I love this product because it is Toshiba and ...
594,54,I bought this laptop Was the worst Laptop I've...
792,252,"So far, a great product. High price tag, howev..."
990,375,This laptop is a great price and has a sleek l...
1188,139,HP Pavilion DV9000 Notebook PC When I fir...
1386,B00KMRGF28_285_AULBCYLXY34W9,Great price and computer! I bought this laptop...


In [15]:
result = translate_texts(df_unduped["text"])

In [14]:
def translate_texts(texts):

    translate_client = translate.Client(credentials=credentials)
    target = 'zh'
    
    texts = list(texts)
    translated_texts = []
    
    result = []
    
    length = len(texts)
    start_pos = 0
    end_pos = 0
    while end_pos < length:
        end_pos += 50
        if end_pos > length:
            end_pos = length
        if end_pos%50 == 0:
            time.sleep(120)
        result.extend(translate_client.translate(
            texts[start_pos:end_pos], target_language=target))
        start_pos = end_pos
#     for text in texts:

#         if isinstance(text, six.binary_type):
#             print(True)
#             text = text.decode('utf-8')

        # Text can also be a sequence of strings, in which case this method
        # will return a sequence of results for each text.
    

#     print(u'Text: {}'.format(result['input']))
#     print(u'Translation: {}'.format(result['translatedText']))
#     print(u'Detected source language: {}'.format(
#         result['detectedSourceLanguage']))
    return result
#     translated_texts.append(result['translatedText'])

#     return translated_texts

In [16]:
orig_texts = []
translated_texts = []

for response in result:
    translated_texts.append(response['translatedText'])
    orig_texts.append(response['input'])

In [17]:
df_unduped["chinese_trans"] = translated_texts

In [18]:
df_unduped["trans_input"] = orig_texts

In [34]:
df_unduped.to_csv("text-level/Test_Gold_Translation.csv", "\t")

In [35]:
df.shape

(15840, 4)

In [19]:
df

,sent_id,polarity,entity-aspect,text
0,348,positive,LAPTOP-GENERAL,Most everything is fine with this machine: spe...
1,348,none,LAPTOP-PRICE,Most everything is fine with this machine: spe...
2,348,positive,LAPTOP-QUALITY,Most everything is fine with this machine: spe...
3,348,positive,LAPTOP-OPERATION_PERFORMANCE,Most everything is fine with this machine: spe...
4,348,none,LAPTOP-USABILITY,Most everything is fine with this machine: spe...
...,...,...,...,...
76225,257,none,COMPANY-USABILITY,I previously owned an HP desktop and a Dell la...
76226,257,none,COMPANY-DESIGN_FEATURES,I previously owned an HP desktop and a Dell la...
76227,257,none,COMPANY-PORTABILITY,I previously owned an HP desktop and a Dell la...
76228,257,none,COMPANY-CONNECTIVITY,I previously owned an HP desktop and a Dell la...


In [20]:
df_unduped

,sent_id,text,chinese_trans,trans_input
0,348,Most everything is fine with this machine: spe...,这台机器的大多数功能都可以满足要求：速度，容量，构造。我唯一不了解的是，对于某些页面（例如Y...,Most everything is fine with this machine: spe...
198,285,"I love the size, keyboard, the functions. I do...",我喜欢尺寸，键盘，功能。我真的没有抱怨。它易于使用，质量好，价格低。完美的三连胜！我会推荐这...,"I love the size, keyboard, the functions. I do..."
396,337,I love this product because it is Toshiba and ...,我喜欢这款产品，因为它是Toshiba及其15.6英寸产品。它没有相机，但是我总是可以轻松地...,I love this product because it is Toshiba and ...
594,54,I bought this laptop Was the worst Laptop I've...,我买的这台笔记本电脑是我买过的最糟糕的笔记本电脑。我花了很多钱在这个产品上，这是一场噩梦。与...,I bought this laptop Was the worst Laptop I've...
792,252,"So far, a great product. High price tag, howev...",到目前为止，很棒的产品。高价格标签，但是。仍在尝试学习如何使用它。,"So far, a great product. High price tag, howev..."
...,...,...,...,...
75240,317,"Ok, this is probably the best laptop series ev...",好的，这可能是苹果有史以来最好的笔记本电脑系列。该表壳是用一块铝雕刻而成的。尽管我选择了最低...,"Ok, this is probably the best laptop series ev..."
75438,B0096VDM8G_5_A2Q6ZHSO5QDPRQ,I just bought this computer from the Apple Sto...,我刚从Apple Store购买了这台计算机，对此印象深刻。它具有惊人的视网膜显示，并且速度...,I just bought this computer from the Apple Sto...
75636,B00KMRGF28_193_A20GPA6RDO8DT7,Meh...but cheap. I bought this so that I could...,嗯...但是便宜。我买了这个东西，以便可以坐在沙发上，用比iPad更大的屏幕浏览互联网。 2...,Meh...but cheap. I bought this so that I could...
75834,391,Let me start with the good:So awesome. Great O...,让我从善良开始：太棒了。出色的操作系统，对现有产品线的出色改进提高了处理器速度，并增加了雷电...,Let me start with the good:So awesome. Great O...


In [21]:
df_chinese = df.merge(df_unduped, on="sent_id") \
               .loc[:,["sent_id", "polarity", "entity-aspect", "chinese_trans"]] \
               .rename(columns = {"chinese_trans":"text"})

In [22]:
df_chinese

,sent_id,polarity,entity-aspect,text
0,348,positive,LAPTOP-GENERAL,这台机器的大多数功能都可以满足要求：速度，容量，构造。我唯一不了解的是，对于某些页面（例如Y...
1,348,none,LAPTOP-PRICE,这台机器的大多数功能都可以满足要求：速度，容量，构造。我唯一不了解的是，对于某些页面（例如Y...
2,348,positive,LAPTOP-QUALITY,这台机器的大多数功能都可以满足要求：速度，容量，构造。我唯一不了解的是，对于某些页面（例如Y...
3,348,positive,LAPTOP-OPERATION_PERFORMANCE,这台机器的大多数功能都可以满足要求：速度，容量，构造。我唯一不了解的是，对于某些页面（例如Y...
4,348,none,LAPTOP-USABILITY,这台机器的大多数功能都可以满足要求：速度，容量，构造。我唯一不了解的是，对于某些页面（例如Y...
...,...,...,...,...
76225,257,none,COMPANY-USABILITY,我以前拥有一台HP台式机和一台Dell笔记本电脑。在拥有它们的第一年内，两台计算机的运行速度...
76226,257,none,COMPANY-DESIGN_FEATURES,我以前拥有一台HP台式机和一台Dell笔记本电脑。在拥有它们的第一年内，两台计算机的运行速度...
76227,257,none,COMPANY-PORTABILITY,我以前拥有一台HP台式机和一台Dell笔记本电脑。在拥有它们的第一年内，两台计算机的运行速度...
76228,257,none,COMPANY-CONNECTIVITY,我以前拥有一台HP台式机和一台Dell笔记本电脑。在拥有它们的第一年内，两台计算机的运行速度...


In [23]:
df_chinese.to_csv("text-level/MTCH_"+"Laptop_Text_Train_NLI_M.csv", sep = "\t", header = False, index = False)

=================================================================================================
translate traditional chinese to simplified and remake a test_gold or dev file in chinese

In [38]:
validated_translation = "./text-level/Test_Gold_Translation_Validated.xlsx"
file_to_tran = "text-level/EN_Laptop_Text_Test_Gold_NLI_M.csv"

In [29]:
df = pd.read_excel(validated_translation, encoding="utf-8")
df = df[["sent_id", "chinese_trans_validated"]]
df.columns = ["sent_id", "text"]

In [31]:
df_unduped = df[["sent_id", "text"]].drop_duplicates(keep = "first")

In [32]:
test_df = df_unduped.iloc[0:8,:]

In [33]:
test_df

,sent_id,text
0,B0074703CM_108_ANONYMOUS,好吧，我的第一台苹果电脑给我留下了深刻的印象。运行良好，快速且无重启。等待安装MS Offi...
1,B00GJUQ4Z0_10_ANONYMOUS,超…級...慢。认真的-真的很慢。无法使用。甚至无法正确阅读。而且-没有俄语输入？搞什麼阿
2,B0146DD02G_18_ANONYMOUS,多么出色的笔记本电脑，我可以运行我的游戏并真正快速地工作。真正的轻便，您可以随身携带。电池寿...
3,B0074703CM_268_ANONYMOUS,物超所值，而且对于寻找Macbook的人来说必须购买。没有比这个更好的价格了，而且它還带有一...
4,B0074703CM_240_ANONYMOUS,从字面上看，它仍然比我的旧Macbook pro酷得多。老式的一体式Macbook pro可...
5,B0074703CM_286_ANONYMOUS,这款Macbook Pro令人赞叹！如果您担心它没有光盘播放器，请不要担心。在没有光盘播放器...
6,B0074703CM_210_ANONYMOUS,我在2012年10月购买了Macbook Pro，两年后，它仍然像以往一样完美无缺！我从来没...
7,B00M166GWM_2_ANONYMOUS,2015年1月从百思买购买了这款笔记本电脑，還是很喜歡。我可以将它用作平板电脑，笔记本电脑或...


In [16]:
result = translate_texts(df_unduped["text"])

In [15]:
def translate_texts(texts):

    translate_client = translate.Client(credentials=credentials)
    target = 'zh'
    
    texts = list(texts)
    translated_texts = []
    
    result = []
    
    length = len(texts)
    start_pos = 0
    end_pos = 0
    while end_pos < length:
        end_pos += 50
        if end_pos > length:
            end_pos = length
        if end_pos%500 == 0:
            time.sleep(120)
        result.extend(translate_client.translate(
            texts[start_pos:end_pos], target_language=target))
        start_pos = end_pos
#     for text in texts:

#         if isinstance(text, six.binary_type):
#             print(True)
#             text = text.decode('utf-8')

        # Text can also be a sequence of strings, in which case this method
        # will return a sequence of results for each text.
    

#     print(u'Text: {}'.format(result['input']))
#     print(u'Translation: {}'.format(result['translatedText']))
#     print(u'Detected source language: {}'.format(
#         result['detectedSourceLanguage']))
    return result
#     translated_texts.append(result['translatedText'])

#     return translated_texts

In [34]:
orig_texts = []
translated_texts = []

for response in result:
    translated_texts.append(response['translatedText'])
    orig_texts.append(response['input'])

In [35]:
df_unduped["simplified_chinese_trans"] = translated_texts

In [36]:
print(df_unduped.loc[df_unduped.sent_id=="B00O0W450A_10_ANONYMOUS",:].values)

[['B00O0W450A_10_ANONYMOUS'
  '我喜欢这台笔记本电脑。它讓我觉得我的旧笔记本电脑相比之下很爛。電持续的时间与要求的一样。我玩模拟游戏大约持續5个小时左右。我还没有正式测试过，但是已经很长时间了。我不想充電笔记本电脑太久而会伤及电池，但是有了这台笔记本电脑，我几乎不需要充电。触摸屏反应灵敏，甚至可以与我笨拙的一美元手写笔配合使用。我唯一的问题是Windows8。我讨厌Windows8。它強迫我登入Microsoft，有时不让我下载对我的计算机有害的内容，但这就是防病毒软件的作用阿。 wifi的谣言还没有发生在我身上。 wifi很好。我认为这是因为我总是按照建议下载Windows更新。感觉質感很好。金属的感觉很贵。关上时，笔记本电脑的厚度略超过一英寸。我是一个非常虚弱的女孩，但可以用一只手提起這笔记本电脑，但是很快我就会累了。 15.6＆＃34;屏幕不适合空手携带。我建议買到一个袋子。该waterfly袋子非常好。'
  '我喜欢这台笔记本电脑。它让我觉得我的旧笔记本电脑相比之下很烂。电持续的时间与要求的一样。我玩模拟游戏大约持续5个小时左右。我还没有正式测试过，但是已经很长时间了。我不想充电笔记本电脑太久而会伤及电池，但是有了这台笔记本电脑，我几乎不需要充电。触摸屏反应灵敏，甚至可以与我笨拙的一美元手写笔配合使用。我唯一的问题是Windows8。我讨厌Windows8。它强迫我登入Microsoft，有时不让我下载对我的计算机有害的内容，但这就是防病毒软件的作用阿。 wifi的谣言还没有发生在我身上。 wifi很好。我认为这是因为我总是按照建议下载Windows更新。感觉质感很好。金属的感觉很贵。关上时，笔记本电脑的厚度略超过一英寸。我是一个非常虚弱的女孩，但可以用一只手提起这笔记本电脑，但是很快我就会累了。 15.6＆＃34;屏幕不适合空手携带。我建议买到一个袋子。该waterfly袋子非常好。']]


In [37]:
df_unduped

,sent_id,text,simplified_chinese_trans
0,B0074703CM_108_ANONYMOUS,好吧，我的第一台苹果电脑给我留下了深刻的印象。运行良好，快速且无重启。等待安装MS Offi...,好吧，我的第一台苹果电脑给我留下了深刻的印象。运行良好，快速且无重启。等待安装MS Offi...
1,B00GJUQ4Z0_10_ANONYMOUS,超…級...慢。认真的-真的很慢。无法使用。甚至无法正确阅读。而且-没有俄语输入？搞什麼阿,超…级...慢。认真的-真的很慢。无法使用。甚至无法正确阅读。而且-没有俄语输入？搞什么阿
2,B0146DD02G_18_ANONYMOUS,多么出色的笔记本电脑，我可以运行我的游戏并真正快速地工作。真正的轻便，您可以随身携带。电池寿...,多么出色的笔记本电脑，我可以运行我的游戏并真正快速地工作。真正的轻便，您可以随身携带。电池寿...
3,B0074703CM_268_ANONYMOUS,物超所值，而且对于寻找Macbook的人来说必须购买。没有比这个更好的价格了，而且它還带有一...,物超所值，而且对于寻找Macbook的人来说必须购买。没有比这个更好的价格了，而且它还带有一...
4,B0074703CM_240_ANONYMOUS,从字面上看，它仍然比我的旧Macbook pro酷得多。老式的一体式Macbook pro可...,从字面上看，它仍然比我的旧Macbook pro酷得多。老式的一体式Macbook pro可...
...,...,...,...
75,B0074703CM_282_ANONYMOUS,我是一名由设计师转为前端开发人员的人员，因此该计算机无法用于前端开发。我有一台量产的PC，与...,我是一名由设计师转为前端开发人员的人员，因此该计算机无法用于前端开发。我有一台量产的PC，与...
76,B00LNFCADQ_5_ALWWKZCL1CKGD,不要浪费你的钱。起初我的HP看起来不错，然后问题就开始了.....无线适配器只会“消失”而不...,不要浪费你的钱。起初我的HP看起来不错，然后问题就开始了.....无线适配器只会“消失”而不...
77,B0074703CM_6_ANONYMOUS,我给这款产品5颗星，因为我一直在寻找MacBook Pro，而且在这里我发现它的价格低廉。我...,我给这款产品5颗星，因为我一直在寻找MacBook Pro，而且在这里我发现它的价格低廉。我...
78,B0074703CM_173_ANONYMOUS,当我决定购买第一台计算机时，我希望自己先购买Mac。如果价格没有吓到您，您会发现它在无故障运...,当我决定购买第一台计算机时，我希望自己先购买Mac。如果价格没有吓到您，您会发现它在无故障运...


In [45]:
df = pd.read_csv(file_to_tran, names = ["sent_id", "polarity", "entity-aspect", "text"], delimiter = "\t")

In [51]:
df_chinese = df.merge(df_unduped, how = "outer", on="sent_id") \
               .loc[:,["sent_id", "polarity", "entity-aspect", "simplified_chinese_trans"]] \
               .rename(columns = {"simplified_chinese_trans":"text"})

In [52]:
#check some individual records
print(df_chinese.loc[df_chinese.sent_id=="B00O0W450A_10_ANONYMOUS",:].values)

[['B00O0W450A_10_ANONYMOUS' 'positive' 'LAPTOP-GENERAL'
  '我喜欢这台笔记本电脑。它让我觉得我的旧笔记本电脑相比之下很烂。电持续的时间与要求的一样。我玩模拟游戏大约持续5个小时左右。我还没有正式测试过，但是已经很长时间了。我不想充电笔记本电脑太久而会伤及电池，但是有了这台笔记本电脑，我几乎不需要充电。触摸屏反应灵敏，甚至可以与我笨拙的一美元手写笔配合使用。我唯一的问题是Windows8。我讨厌Windows8。它强迫我登入Microsoft，有时不让我下载对我的计算机有害的内容，但这就是防病毒软件的作用阿。 wifi的谣言还没有发生在我身上。 wifi很好。我认为这是因为我总是按照建议下载Windows更新。感觉质感很好。金属的感觉很贵。关上时，笔记本电脑的厚度略超过一英寸。我是一个非常虚弱的女孩，但可以用一只手提起这笔记本电脑，但是很快我就会累了。 15.6＆＃34;屏幕不适合空手携带。我建议买到一个袋子。该waterfly袋子非常好。']
 ['B00O0W450A_10_ANONYMOUS' 'none' 'LAPTOP-PRICE'
  '我喜欢这台笔记本电脑。它让我觉得我的旧笔记本电脑相比之下很烂。电持续的时间与要求的一样。我玩模拟游戏大约持续5个小时左右。我还没有正式测试过，但是已经很长时间了。我不想充电笔记本电脑太久而会伤及电池，但是有了这台笔记本电脑，我几乎不需要充电。触摸屏反应灵敏，甚至可以与我笨拙的一美元手写笔配合使用。我唯一的问题是Windows8。我讨厌Windows8。它强迫我登入Microsoft，有时不让我下载对我的计算机有害的内容，但这就是防病毒软件的作用阿。 wifi的谣言还没有发生在我身上。 wifi很好。我认为这是因为我总是按照建议下载Windows更新。感觉质感很好。金属的感觉很贵。关上时，笔记本电脑的厚度略超过一英寸。我是一个非常虚弱的女孩，但可以用一只手提起这笔记本电脑，但是很快我就会累了。 15.6＆＃34;屏幕不适合空手携带。我建议买到一个袋子。该waterfly袋子非常好。']
 ['B00O0W450A_10_ANONYMOUS' 'positive' 'LAPTOP-QUALITY'
  '我喜欢这台笔记本电脑。它让我觉得我的旧笔记本电脑相比之下很烂。电持

In [60]:
#check line alignment and total record count
(df_chinese.sent_id == df.sent_id).sum()

15840

In [61]:
df_chinese.to_csv("text-level/CH_"+"Laptop_Text_Test_Gold_NLI_M.csv", sep = "\t", header = False)